In [ ]:
!pip install --upgrade google-cloud-pubsub
!pip install --upgrade 'apache-beam[gcp]'
!pip install --upgrade google-apitools


In [ ]:
PROJECT_ID = %env GOOGLE_CLOUD_PROJECT

In [ ]:
import apache_beam as beam
import sys
import time
from time import Duration

from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions

sys.argv = ["11_tumbling_window.ipynb"]
encoding = "utf-8"
input_subscription = "projects/{}/subscriptions/ratings-sub".format(PROJECT_ID)

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True

p = beam.Pipeline(options=options)

rating_count = "projects/{}/topics/ratings_count_window".format(PROJECT_ID)


def format_output(r):
    (movie_id, rating) = r
    return "{r} ratings for movieId {id}".format(r=rating, id=movie_id)


pubsub_pipeline = (
    p
    | "Read from PubSub topic" >> beam.io.ReadFromPubSub(subscription=input_subscription)
    | "Split the records by comma" >> beam.Map(lambda row: row.decode(encoding).split(","))
    | "With Custom timestamp" >> beam.Map(lambda row: beam.window.TimestampedValue(row, time.time()))
    | "Form KV pair" >> beam.Map(lambda r: (r[1], float(r[2])))
    # 1 seconds
    | "Window" >> beam.WindowInto(beam.window.FixedWindows(20), allowed_lateness=Duration(1))
    # | "Allowing lateness" >> beam.WindowInto(beam.window.FixedWindows(20), allowed_lateness=Duration(1*24*60*60)) # 1 day
    | "Count the ratings" >> beam.transforms.combiners.Count.PerKey()
    | "Format output" >> beam.Map(format_output)
    | "Write to PubSub" >> beam.io.WriteStringsToPubSub(rating_count)
)

result = p.run()
result.wait_until_finish()


ERROR:apache_beam.runners.direct.executor:Exception at bundle <apache_beam.runners.direct.bundle_factory._Bundle object at 0x1254a0f48>, due to an exception.
 Traceback (most recent call last):
  File "/opt/venv/lib/python3.7/site-packages/google/api_core/grpc_helpers.py", line 67, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "/opt/venv/lib/python3.7/site-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/opt/venv/lib/python3.7/site-packages/grpc/_channel.py", line 849, in _end_unary_response_blocking
    raise _InactiveRpcError(state)
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.DEADLINE_EXCEEDED
	details = "Deadline Exceeded"
	debug_error_string = "{"created":"@1634320983.763516000","description":"Error received from peer ipv6:[2800:3f0:4002:80e::200a]:443","file":"src/core/lib/surface/call.cc","file_line":1070,"grpc_message":"D